In [1]:
import asyncio
import aiohttp
from itertools import product
from os import walk

nome_saida = 'Arthur_IPTV_limitada.m3u'

linhas = []
for item in walk('.'):
    for i in item[-1]:
        if i in [nome_saida, 'IPTV Filter.ipynb']:
            continue
#         print(i)
        with open(i, 'r') as f:
            linhas.extend(f.readlines())
    break

urls = []
metadata = dict()
extensoes = ['.mp4' ,'.flv', '.f4v', '.wmv', 'avi', 'youtu', 'fb.', 'facebook', 
             'Iptvvictor_Addon', 'srregio.com']
for i, linha in enumerate(linhas):
    linha = linha.strip()
    if linha and linha[0] != '#':
        if any(ext in linha for ext in extensoes):
            continue
        urls.append(linha)
        if i-1 >= 0 and linhas[i-1][0] == '#':
            mdados = metadata.get(linha, set())
            mdados.add(linhas[i-1])
            metadata[linha] = mdados  

# urls = set([l.strip() for l in linhas if l[0] != '#' and l.strip() != ''])

In [2]:
urls = set(urls)
rejeitos_parciais = ['http:// ', 'http://\n', '0.0.0.0', 'whatsup', ' 9']
[[urls.remove(url) for url in set(urls) if r in url] for r in rejeitos_parciais]

rejeitos = ['http://']
[urls.remove(r) for r in rejeitos]

# for m in metadata:
#     if m not in urls:
#         continue
#     if len(metadata[m]) > 1:
#         print('\n', m)
#         [print('\t', d.strip()) for d in metadata[m]]

urls = list(urls)

In [3]:
HEADERS = {'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/45.0.2454.101 Safari/537.36')}

async def get_response(url):
    async with aiohttp.ClientSession() as session:
        response = None
        try:
            response = await session.get(url, headers=HEADERS)
        except Exception as e:
#             print(url, e)
            pass
        else:
            pass
#             print(url, response.status)
        return response, url

done_results = []
def done(fut):
    done_results.append(fut.result())


futures = [asyncio.ensure_future(get_response(url)) 
               for url in urls]

[f.add_done_callback(done) for f in futures]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
# loop.close()

print(len(futures))

5469


In [81]:
substituicoes = {
    'TV ABERTA':'TV Aberta',
    'CANAL ABERTO':'TV Aberta',
    'CANAIS TV ABERTA':'TV Aberta',
    'CANAIS ABERTOS':'TV Aberta',
    'WEB TV':'TV Aberta',
    'WEBTV':'TV Aberta',
    'ABERTOS':'TV Aberta',
    'BAND': 'TV Aberta',
    'GOIÁS': 'TV Aberta',
    'BAHIA': 'TV Aberta',
    'SÃO PAULO': 'TV Aberta',
    'ESPÍRITO SANTO': 'TV Aberta',
    'ALAGOAS': 'TV Aberta',
    'DISTRITO FEDERAL':'TV Aberta',
    
    'RADIO':'RÁDIO & MÚSICA',
    'RÁDIO':'RÁDIO & MÚSICA',
    'RÁDIOS':'RÁDIO & MÚSICA',
    'RADIOS':'RÁDIO & MÚSICA',
    'SOUND!':'RÁDIO & MÚSICA',
    'MÚSICA':'RÁDIO & MÚSICA',
    'MUSICA':'RÁDIO & MÚSICA',
    'MUSIC':'RÁDIO & MÚSICA',
    'RÁDIO & MÚSICA':'RÁDIO & MÚSICA',
    'VARIEDADE E MÚSICAS':'RÁDIO & MÚSICA',
    'ANTENA':'RÁDIO & MÚSICA',
    'ANTENNA':'RÁDIO & MÚSICA',
    
    'CAMERAS':'CÂMERAS',
    
    'CANAIS PORNO':'ADULTO',
    'ADULTOS':'ADULTO',
    
    'DEPORTES':'ESPORTE',
    'DESPORTES':'ESPORTE',
    'SPORTS':'ESPORTE',
    'SPORT':'ESPORTE',
    'HD SPORTS':'ESPORTE',
    'ESPORTES INTERNACIONAIS':'ESPORTE',
    'ESPORTES':'ESPORTE',
    
    'NOTICIAS':'NOTÍCIAS',
    'ABERTAS-NOTICIAS':'NOTÍCIAS',
    'JORNALISMO':'NOTÍCIAS',
    'NEWS':'NOTÍCIAS',
    
    #     '':'TV Aberta',
}


metadados_rejeito = ['A FEITICEIRA', 'FILMES BR', 'FILMES LANÇAMENTOS', 'BBB18',
                     'RELIGIOSO', 'Gospel', 'TODO MUNDO ODEIA', 'TRAILERS', 'JUDÁ',
                     'CATOLICO', 'CATÓLICO', 'CATHOLIC', 'GOLPEL', 'EVANGÉLICOS', 
                     "WEB'S TV CLIPES", 'FUTEBOL', 'REALITY', 'RELIGION']

preferidos = ['TV Aberta']
preteridos = ['#Extgrp:Premium Spain', '#Extgrp:Adultos']

In [94]:
len(done_results)
OK = [str(original) for r, original in done_results if r and r.status == 200]

def cut(string, head, upper):
    n_string = ''
    if string in head:
        idx = head.index(string)
        if '"' in head[idx:]:
            n_string = head[idx:].split('"')[1]
        else:
            n_string = head[idx:].split('=')[1]
        if upper: n_string = n_string.upper()
    return n_string.strip()

print(len(OK))

urls_ok = []
for url in metadata:
    
    if url not in OK:
        #print('Erro no link:', url)
        continue
    
    metadados = metadata.get(url)
    #print(url, metadados)
    if not metadados:
        urls_ok.append('#EXTINF:-1, group-title="ZZ DESCONHECIDO", "DESCONHECIDO"\n'
                       + url)
    else:
        groups, logos, titulos= [],[],[]
        
        for metadado in metadados:
            
            g = cut('group-title', metadado, True)
            if g: groups.append(substituicoes.get(g, g)) 
            
            l = cut('tvg-logo', metadado, False)
            if l: logos.append(l)
            
            t = metadado.split(',')[-1].strip().title()
            if t.strip(): titulos.append(t)

        if not groups:
            for t in titulos:
                for chute in substituicoes:
                    if chute in t.upper():
                        groups.append(substituicoes[chute])
                        break
        
        if any(p in groups for p in preferidos):
            groups = [p for p in preferidos if p in groups]
        
        if len(groups) > 1:
            groups = [g for g in groups if g not in preteridos]
        
        if len(titulos) > 1:
#             print(titulos)
            titulos = [t for t in titulos if t not in preteridos]
        
        
        groups = set(groups or ['ZZ DESCONHECIDO'])
        titulos = set(titulos or ['DESCONHECIDO'])
        logos = set(logos or [''])      
        
        # TODO remover os demais quando houver uma opção preferida                        
        
        cabecalho=''
        for g,l,t in set(product(groups, logos, titulos)):
            if g and t:
                cabecalho += \
                    f'#EXTINF:-1 group-title="{g}" tvg-id="{t}" tvg-logo="{l}", {t}\n' 
            else:
                print(g,t,sep='|')
            
        if len(cabecalho.split('\n')) > 2:    
            cabecalho = '#!\n' + cabecalho
        
        doit = True
        for rejeito in metadados_rejeito:
            if rejeito in cabecalho:
                doit = False
                break
        
        
        
        if doit: urls_ok.append(cabecalho + url + '\n')
        


1786


In [95]:
with open(nome_saida, 'w') as f:
    print('#EXTM3U\n\n', file=f)
    for url in sorted(urls_ok):
        print(url, file=f)